02 特徴選択のための単語除去
======================

* `Bag-of-Words`は文章中に含まれる単語を元に特徴量を生成するが、全ての単語が有用な特徴量となる訳ではない

    * そこで、特徴量の数を削減するために、有用な特徴量になりそうにならない単語をあらかじめ削除するという方法が取られる
    
* ここでは、単語から作られる特徴量に対するフィルタ法について説明する

## 1. ストップワードによる単語除去

* テキスト分類や情報探索などのタスクでは、文章に対する深い理解を必要としない

    * そのため、代名詞、冠詞、前置詞のような、文章の理解によらず一般的に使われる単語を特徴量に加えることにあまり意味はない
    
    * このような単語は`ストップワード`と呼ばれ、処理の対象外とするのが一般的

* Pythonの有名な自然言語処理パッケージ`NLTK`には、様々な言語に対して言語学者が定義したストップワードが定義されている

    * 例)a, about, above, am, an...
    
* このリストに含まれるストップワードは、アポストロフィ(')を含み、全て小文字で統一されていることに注意する

    * もしトークン化でアポストロフィを区切り文字として使用していた場合、このリストをそのまま利用することはできない
    
    * また、ストップワードかどうかを判定するコードを書くときに、大文字と小文字を区別していると、ストップワードを認識できない

## 2. 頻度に基づく単語除去

### 1. 頻出単語の除去

* ストップワードを除去することで、不要と思われる特徴量の生成を抑えることができる

* しかし、ストップワードのリストは汎用性を目的として作成されているため、特定のコーパス(解析対象となる文書全体)が持つ事情をうまく捉えられない

    * 通常は有用な単語でも、特定のコーパスでは有用でない単語が存在する
    
        * 例)New York TImes における"New York TImes"
    
    * このような単語を見つけ出すために、頻度情報が利用できる

* 単語の出現頻度を調べることで、コーパスに特有の除去すべき単語を浮き彫りにできる

    * 例)以下の表のYelpレビューデータセットにおける頻出単語の上位40個
    
        * この表には多くのストップワードが含まれる
        
        * この表のストップワード以外の単語が、コーパスに特有の除去すべき単語

In [1]:
## ここにPandasで表を作る
## 上位40個数まで

* 頻度に基づく単語除去は実用的で役に立つが、使用する際には上位何個までを除去するかを決める必要がある

    * これを見つけるのは難しい問題であり、ほとんどの場合は自動で決定する方法はない
    
    * また、データセットが変わると再検討する必要がある

### 2. レアな単語の取り扱い

* タスクによっては、「レアな単語」を除去する必要がある

    * これは、出現頻度の低い単語のこと
    
    * 本当に使用頻度が少ない単語の場合もあれば、単なるスペルミスの場合もある
    
    * 機械学習にとって、1つか2つの文書にしか現れない単語はノイズでしかない
    
* 例)Yelpデータセットにおいて、レビューに基づいて店舗のカテゴリを分類するタスクを考える

    * 1つのレビューに"gobbledygook"という単語が含まれていたとき、これは分類できない
    


* レアな単語が実際のデータセットにどれくらい存在するかを確認する

    * Yelpデータセットには約160万件のレビュー文書が含まれる
    
    * これをスペースと句読点を区切り文字としてトークン化すると、単語数(語彙数)は357,481個となる
    
    * このうち、1つの文書にしか登場しない単語は189,915個、2つの文書にしか登場しない単語は41,162個となる
    
        * 合計すると、語彙の60%以上がレアな単語となる(これらは除去する)

* レアな単語は除去する前に、まとめて1つの特徴量として扱うこともできる

    * 以下の図に例を示す
    
    * この図では、"gobbledygook"と"zylophant"という2つのレアな単語が含まれるテキストを`Bag-of-Words`に変換している
    
    * その際、レアな単語を1つにまとめてGARBAGEという名前の特徴量としている
    
    * この方法は、ストップワードや頻度に基づく単語除去と問題なく組み合わせて使用できる
    
    * 注意点として、コーパス全体のカウント処理が終わるまでどれがレアな単語かわからないため、GARBAGE特徴量を作成する前に、普通の`BoW`を完成させる
    
![GARBAGE特徴量を利用したBag-of-Words表現](./images/GARBAGE特徴量を利用したBag-of-Words表現.png)

* 特徴量(単語)にフォーカスしているが、一方でデータ点にも同様のものが存在する

    * 例)Wikipediaダンプデータには不完全な`スタブ`(説明不十分項目)が多く含まれる
    
    * テキスト文書が非常に短い場合、有益な情報を含まないため、学習モデルにとってノイズとなる
    
        * そのため、短い文章は学習データから取り除いた方が良い
        
        * ただし、ツイートのようにもともと短いテキストに対しては、特徴量生成とモデリングに別の手法を使う必要がある

### 3. ステミング(語幹処理)

* トークン化された単語をそのまま使うと、同じ単語の変化形が別の単語としてカウントされてしまうという問題がある

    * 例)"flower"と"flowers"は単数形か複数形かの違いだけなのに、別の単語としてカウントされる
    
    * 例)"swimmer"、"swim"は非常に意味が近いのに、異なる単語として識別される
    


* `ステミング`：単語を語幹の形に変換する自然言語処理の技術

    * 語学的なルールに基づくものもあれば、統計量に基づくものもある
    
    * アルゴリズムの中には、品詞タグ付けと見出し語化と呼ばれる語学的ルールが組み込まれているものもある

* ほとんどの`ステミングツール`は、英語のみ対応している

    * 英語に対するステミングを行うフリーツールとして最も広く使われているのが、Porter Stemmer
    
* Pythonの`NLTKパッケージ`を使ってPorter Stemmerを実行するコードを下に示す

    * ただし、完璧ではない

In [2]:
import nltk
stemmer = nltk.stem.porter.PorterStemmer()

stemmer.stem('flowers')

'flower'

In [3]:
stemmer.stem('zeroes')

'zero'

In [4]:
stemmer.stem('stemmer')

'stemmer'

In [5]:
stemmer.stem('sixties')

'sixti'

In [6]:
stemmer.stem('sixty')

'sixti'

In [7]:
stemmer.stem('goes')

'goe'

In [8]:
stemmer.stem('go')

'go'

* `ステミング`には計算コストがかかる

    * このコストが利益を上回るかどうかはタスクによる
    
    * また、ステミングを行うことで本当は異なる単語を同じにしてしまうというデメリットもある
    
    * 例)"new"と"news"は本来異なる意味

| 版   | 年/月/日   |
| ---- | ---------- |
| 初版 | 2019/04/21 |